In [1]:
import pandas as pd
from mapbox import Geocoder

In [2]:
geocoder = Geocoder(access_token="pk.eyJ1IjoiamVycnlsaW5ldyIsImEiOiJjajIwc2E1YXkwMmt6MzNuMXZnaWRjb2lhIn0.u0AHZNQd8_8hFmpaVvI1nQ")

In [9]:
df = pd.read_csv("other_westwood_ppp_small.csv")
df

,LoanRange,BusinessName,Address,JobsRetained
0,$5-10 million,"GREAT AMERICAN CHICKEN CORP, INC.","1990 S Bundy Dr., Suite 250",0.0
1,$5-10 million,WEST COAST DENTAL SERVICES INC,12121 Wilshire Blvd,0.0
2,$2-5 million,"ASPIRIANT, LLC","11100 Santa Monica Blvd., Suite 600",188.0
3,$2-5 million,"BEVERLY RADIOLOGY MEDICAL GROUP, INC.",1510 Cotner Avenue,163.0
4,$2-5 million,"BHP HOLDING PARTNERS, LTD","2034 Cotner Avnue, 4th Floor",140.0
...,...,...,...,...
371,"$150,000-350,000","WIZ KHALIFA TOURING, INC.",11766 Wilshire Blvd Suite 500,13.0
372,"$150,000-350,000","WOLFF URBAN MANAGEMENT, INC.",11828 LA GRANGE AVE,14.0
373,"$150,000-350,000","WURRLY, LLC",12301 Wilshire Blvd. Suite 303,13.0
374,"$150,000-350,000","YARD DOG TV, INC",2023 S. WESTGATE AVE,25.0


In [4]:
a = df.iloc[0]["Address"]
a

'1990 S Bundy Dr., Suite 250'

In [5]:
response = geocoder.forward(a).json()
response

{'type': 'FeatureCollection',
 'query': ['1990', 's', 'bundy', 'dr', 'suite', '250'],
 'features': [{'id': 'address.5868995637558794',
   'type': 'Feature',
   'place_type': ['address'],
   'relevance': 1,
   'properties': {'accuracy': 'point'},
   'text': 'South Bundy Drive',
   'place_name': '1990 South Bundy Drive, Los Angeles, California 90025, United States',
   'center': [-118.45559, 34.03419],
   'geometry': {'type': 'Point', 'coordinates': [-118.45559, 34.03419]},
   'address': '1990',
   'context': [{'id': 'neighborhood.292400', 'text': 'Sawtelle'},
    {'id': 'postcode.5149060965450170', 'text': '90025'},
    {'id': 'place.7397503093427640', 'wikidata': 'Q65', 'text': 'Los Angeles'},
    {'id': 'region.9803118085738010',
     'wikidata': 'Q99',
     'short_code': 'US-CA',
     'text': 'California'},
    {'id': 'country.19678805456372290',
     'short_code': 'us',
     'wikidata': 'Q30',
     'text': 'United States'}]},
  {'id': 'address.5614280621307076',
   'type': 'Feature'

In [10]:
df['long'] = df.apply(
     lambda row: 
         geocoder.forward(row.Address+", Los Angeles CA 90025").json()['features'][0]['center'][0], axis=1)
df['lat'] = df.apply(
     lambda row: 
         geocoder.forward(row.Address+", Los Angeles CA 90025").json()['features'][0]['center'][1], axis=1)
df

,LoanRange,BusinessName,Address,JobsRetained,long,lat
0,$5-10 million,"GREAT AMERICAN CHICKEN CORP, INC.","1990 S Bundy Dr., Suite 250",0.0,-118.455590,34.034190
1,$5-10 million,WEST COAST DENTAL SERVICES INC,12121 Wilshire Blvd,0.0,-118.468161,34.043857
2,$2-5 million,"ASPIRIANT, LLC","11100 Santa Monica Blvd., Suite 600",188.0,-118.444651,34.047255
3,$2-5 million,"BEVERLY RADIOLOGY MEDICAL GROUP, INC.",1510 Cotner Avenue,163.0,-118.447760,34.049260
4,$2-5 million,"BHP HOLDING PARTNERS, LTD","2034 Cotner Avnue, 4th Floor",140.0,-118.440960,34.041376
...,...,...,...,...,...,...
371,"$150,000-350,000","WIZ KHALIFA TOURING, INC.",11766 Wilshire Blvd Suite 500,13.0,-118.462320,34.048030
372,"$150,000-350,000","WOLFF URBAN MANAGEMENT, INC.",11828 LA GRANGE AVE,14.0,-118.452764,34.035424
373,"$150,000-350,000","WURRLY, LLC",12301 Wilshire Blvd. Suite 303,13.0,-118.469483,34.042706
374,"$150,000-350,000","YARD DOG TV, INC",2023 S. WESTGATE AVE,25.0,-118.452930,34.034980


In [11]:
df.to_csv("other_westwood_ppp_small_cleaned.csv", index=False)

In [1]:
from json import load, JSONEncoder
from argparse import ArgumentParser, FileType
from re import compile
import sys

from shapely.geometry import Polygon, mapping
from shapely.ops import cascaded_union

In [6]:

file = {
    "type": "FeatureCollection",
    "features": [    
        {
            "type": "Feature",
            "properties": {
                "kind": "L.A. County Neighborhood (Current)",
                "external_id": "athens",
                "name": "Athens",
                "slug": "athens-la-county-neighborhood-current",
                "set": "/1.0/boundary-set/la-county-neighborhoods-current/",
                "metadata": {"sqmi": 1.33275332251, "type": "unincorporated-area", "name": "Athens", "slug": "athens"},
                "resource_uri": "/1.0/boundary/athens-la-county-neighborhood-current/"
            },
            "geometry": { "type": "Polygon", "coordinates": [ 
                        [ -118.291663972105155, 33.930963005656039 ], 
                        [ -118.291524972104938, 33.930962005655957 ], 
                        [ -118.291606982357763, 33.916422006214361 ], 
                        [ -118.313912980479031, 33.916452000075033 ], 
                        [ -118.31398199285357, 33.918361003810489 ], 
                        [ -118.315209985025277, 33.918433998438147 ], 
                        [ -118.316483987315152, 33.917656005465453 ], 
                        [ -118.31778797898059, 33.917520003156469 ], 
                        [ -118.317768979862549, 33.921606005763273 ], 
                        [ -118.313390992418448, 33.9234099992319 ], 
                        [ -118.313391986058718, 33.93091200460988 ], 
                        [ -118.291663972105155, 33.930963005656039 ] 
                ] }
        },
        {
            "type": "Feature",
            "properties": {
                "kind": "L.A. County Neighborhood (Current)",
                "external_id": "westmont",
                "name": "Westmont",
                "slug": "westmont-la-county-neighborhood-current",
                "set": "/1.0/boundary-set/la-county-neighborhoods-current/",
                "metadata": {"sqmi": 1.84381583493, "type": "unincorporated-area", "name": "Westmont", "slug": "westmont"},
                "resource_uri": "/1.0/boundary/westmont-la-county-neighborhood-current/"
            },
            "geometry": { "type": "Polygon", "coordinates": [  
                [ -118.292472989872849, 33.95951400322592 ], 
                [ -118.291595982569675, 33.959495004635755 ], 
                [ -118.291524972104938, 33.930962005655957 ], 
                [ -118.317752979089761, 33.930901001647982 ], 
                [ -118.317719982513182, 33.94546299957149 ], 
                [ -118.313371983162199, 33.945460004723991 ], 
                [ -118.313386989551688, 33.938214000142573 ], 
                [ -118.30902499719825, 33.938210004721995 ], 
                [ -118.309014974916209, 33.941833000143951 ], 
                [ -118.305190995504958, 33.941845002911577 ], 
                [ -118.305179974908896, 33.945467000185211 ], 
                [ -118.302453995362868, 33.945474003461399 ], 
                [ -118.302447974911544, 33.949993000109373 ], 
                [ -118.300261995361026, 33.949999003468804 ], 
                [ -118.300266974796259, 33.959481001336009 ], 
                [ -118.292472989872849, 33.95951400322592 ] 
             ] }
        }
    ]
}

polygons = []

for feat in file['features']:
    polygon = Polygon([ (coor[0], coor[1]) for coor in  feat['geometry']['coordinates'] ])
    polygons.append(polygon)

new_geometry = mapping(cascaded_union(polygons)) # This line merges the polygones

new_feature = dict(type='Feature', id="", properties=dict(Name=""),geometry=dict(type=new_geometry['type'], coordinates=new_geometry['coordinates']))

outjson = dict(type='FeatureCollection', features=[new_feature])

encoder = JSONEncoder(separators=(',', ':'))
encoded = encoder.iterencode(outjson)

# output = outfile

for token in encoded:
    print(token)

{
"type"
:
"FeatureCollection"
,
"features"
:
[
{
"type"
:
"Feature"
,
"id"
:
""
,
"properties"
:
{
"Name"
:
""
}
,
"geometry"
:
{
"type"
:
"Polygon"
,
"coordinates"
:
[
[
[-118.29152497210494
,33.93096200565596
]
,
[-118.29160698235776
,33.91642200621436
]
,
[-118.31391298047903
,33.91645200007503
]
,
[-118.31398199285357
,33.91836100381049
]
,
[-118.31520998502528
,33.91843399843815
]
,
[-118.31648398731515
,33.91765600546545
]
,
[-118.31778797898059
,33.91752000315647
]
,
[-118.31776897986255
,33.92160600576327
]
,
[-118.31339099241845
,33.9234099992319
]
,
[-118.31339198594485
,33.93091114492958
]
,
[-118.31775297908976
,33.93090100164798
]
,
[-118.31771998251318
,33.94546299957149
]
,
[-118.3133719831622
,33.94546000472399
]
,
[-118.31338698955169
,33.93821400014257
]
,
[-118.30902499719825
,33.938210004721995
]
,
[-118.30901497491621
,33.94183300014395
]
,
[-118.30519099550496
,33.94184500291158
]
,
[-118.3051799749089
,33.94546700018521
]
,
[-118.30245399536287
,33.9454740034614

In [7]:
import json

with open('the-stack/test.geojson', 'r') as f:
    data = json.load(f)

#A Python dictionary containing properties to be added to each GeoJSON Feature
properties_dict={
    "cases": None,
    "population": None
    }
#Convert the dictionary to a list
# properties_list=zip(properties_dict.keys(),properties_dict.values())

#Loop over GeoJSON features and add the new properties
for feat in data['features']:
#     for i in range(len(properties_list)):
    feat['properties'].update(properties_dict)
#         feat ['properties'][properties_list[i][0]]=properties_list[i][1]

#Write result to a new file
with open('the-stack/output.geojson', 'w') as f:
    json.dump(data, f, indent=4)

FileNotFoundError: [Errno 2] No such file or directory: 'the-stack/test.geojson'

In [6]:
import csv, json
from geojson import Feature, FeatureCollection, Point

features = []
with open('westwood_ppp_small_cleaned.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for LoanRange, BusinessName, Address, JobsRetained, long, lat in reader:
        print(long)
        print(lat)
        latitude, longitude = map(float, (lat, long))
        features.append(
            Feature(
                geometry = Point((longitude, latitude)),
                properties = {
                    'loan': LoanRange,
                    'name': BusinessName,
                    'jobsRetained': JobsRetained
                }
            )
        )

collection = FeatureCollection(features)
with open("GeoObs.json", "w") as f:
    f.write('%s' % collection)

long
lat


ValueError: could not convert string to float: 'lat'